In [2]:
import pandas as pd
import numpy as np
import math 

#load questions csv
questions_data = pd.read_csv('../OkCupidData/question_data.csv',header=0,delimiter=';' )
questions_data = questions_data.rename(columns={questions_data.columns[0]: "q" })
#keep only question lines
questions_data = questions_data[questions_data.q.str[0] == 'q']
#sort them by non empty number of lines
questions_data.sort_values(by=['N'],ascending=False)
orig_data = pd.read_pickle("../OkCupidData/unprocessed.bz2")

prob_data = orig_data[[col for col in orig_data.columns if col.startswith('p')]]
question_data = orig_data[[col for col in orig_data.columns if col.startswith('q')]]
demog_data = orig_data[[col for col in orig_data.columns if not col.startswith('q') and not col.startswith('p')]]
demog_question_data = orig_data[[col for col in orig_data.columns if not col.startswith('p')]]

In [21]:
#these dimensions are complementary, need to swap them 
indx_p_dominant = prob_data.columns.get_loc("p_dominant")
indx_p_submissive = prob_data.columns.get_loc("p_submissive")

def user_similarity(user_1_indx, user_2_indx, prob_data=prob_data):
    values_1 = prob_data.iloc[[user_1_indx]].values[0]
    values_2 = prob_data.iloc[[user_2_indx]].values[0]
    print values_2[indx_p_dominant], values_2[indx_p_submissive]
    values_2[indx_p_dominant], values_2[indx_p_submissive] =  values_2[indx_p_submissive], values_2[indx_p_dominant] 
    print values_2[indx_p_dominant], values_2[indx_p_submissive]
    
    sum_sq_1 = np.nansum(values_1*values_1)
    if sum_sq_1 == 0:
        return 0
    sum_sq_2 = np.nansum(values_2*values_2)
    if sum_sq_2 == 0:
        return 0
    return np.nansum(values_1*values_2) / (np.sqrt(sum_sq_1) * np.sqrt(sum_sq_2))

user_similarity(2711, 304)

nan nan
nan nan


0

In [8]:
def test_similarity_metric():
    #find closest vector
    max_val = 0
    for i in range(len(prob_data)):
        cosine_sim = user_similarity(2711, i)
        if cosine_sim > max_val:
            print cosine_sim
            max_val = cosine_sim
            display(prob_data.iloc[[i]])
# test_similarity_metric()
prob_data.columns

Index([u'p_conf', u'p_laidback', u'p_drug', u'p_lit', u'p_progress',
       u'p_roman', u'p_dominant', u'p_polit', u'p_pure', u'p_manners',
       u'p_submissive', u'p_inde', u'p_kinky', u'p_opti', u'p_sloppy',
       u'p_indie', u'p_introvert', u'p_arro', u'p_ambi', u'p_cool',
       u'p_trusting', u'p_thrift', u'p_organ', u'p_sexdrive', u'p_honest',
       u'p_expsex', u'p_artsy', u'p_scien', u'p_spon', u'p_explife',
       u'p_greed', u'p_capi', u'p_giving', u'p_explove', u'p_convenmoral',
       u'p_aggre', u'p_kind', u'p_extro', u'p_friendstrangers', u'p_oldfash',
       u'p_comp', u'p_lovedri', u'p_sprit', u'p_passion', u'p_energetic',
       u'p_exer', u'p_logic', u'p_math', u'p_geeky', u'p_adven'],
      dtype='object')